In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('/home/dtward/data/csh_data/emlddmm')
sys.path.append('/home/abenneck/Desktop/emlddmm')
import emlddmm

from scipy.ndimage import distance_transform_edt
import time

user = 'abenneck'

## Step 0: Load data and plot 2 of the (x,y,z) coordinates with 1 property as a heatmap

In [ ]:
# Load data
pos_and_metric_path = '/home/abenneck/mounts/bmaproot/nafs/dtward/andrew_work/test/dragonfly_work/morpho_and_dist_Nov2023.csv'
data = pd.read_csv(pos_and_metric_path)

xcoords = data['x']
ycoords = data['y']
zcoords = data['z']

x = list(zip(ycoords, zcoords))

prop_name = 'D(brainBound)'
prop = data[prop_name]

data.columns

In [ ]:
fig, ax = plt.subplots()

# ax.grid(True)
h = ax.scatter(ycoords, zcoords, c = prop)
plt.colorbar(h)

ax.set_xlabel('x0 = y')
ax.set_ylabel('x1 = z (Dorsal => Ventral)')
ax.set_title(prop_name)


## Load the Atlas

In [ ]:
ontology_name = '/home/abenneck/nafs/dtward/dong/upenn_atlas/atlas_info_KimRef_FPbasedLabel_v2.7.csv'
seg_name = '/home/abenneck/nafs/dtward/dong/upenn_atlas/UPenn_labels_reoriented_origin.vtk'
orientation = 'W'

# Load Atlas coordinates
xS,S,_,_ = emlddmm.read_data(seg_name)

atlas_x = xS[0]
atlas_y = xS[1]
atlas_z = xS[2]

In [ ]:
# Round neuron coord to nearest atlas coord
def roundCoord(sub_xS, coord):
    if coord < np.min(sub_xS):
        coord = np.min(sub_xS)
    elif coord > np.max(sub_xS):
        coord = np.max(sub_xS)
    else:
        coord_rounded = 20*round(coord/20) # Round coordinate to nearest 20
        if coord < 0:
            if coord - coord_rounded >= 0:
                coord = coord_rounded + 10
            else:
                coord = coord_rounded - 10
        else:
            if coord - coord_rounded >= 0:
                coord = coord_rounded + 10
            else:
                coord = coord_rounded - 10
    return coord

In [ ]:
# Returns a mask where regional boundary voxels are 1 and all other voxels are 0
def generate_regional_boundary_mask(S):
    S_bound = np.zeros_like(S)
    
    for voxelID in np.unique(S)[1:]:
        start = time.time()
        print(f'({np.where(np.unique(S)[1:] == voxelID)[0][0]}/{len(np.unique(S)[1:])}) Starting region {voxelID}')
        
        S_temp = np.zeros_like(S)
        S_temp[S == voxelID] = 1 # Sets all voxels labelled as 'voxelID' to 1, and all other voxels remain 0
        
        boundary_mask = distance_transform_edt(S_temp)  # Computes distance transform of subregion
        boundary_mask[boundary_mask != 1.0] = 0.0  # Only boundary pixels have a value of 1.0 (pixels inside > 1.0 and pixels outside = 0.0)
        
        S_bound = np.logical_or(S_bound,boundary_mask == 1.0)
        
        print(f'Finished region {voxelID} in {time.time() - start}s')
    
    np.savez('/home/abenneck/nafs/dtward/andrew_work/test/dragonfly_work/S_bound.npz', mask = S_bound)

# generate_regional_boundary_mask(S)

# Example in 2D

## Step 1: Define a voxel grid

In [ ]:
xmin = np.min(xcoords)
xmax = np.max(xcoords)

ymin = np.min(ycoords)
ymax = np.max(ycoords)

zmin = np.min(zcoords)
zmax = np.max(zcoords)

print(f'xrange: ({xmin}, {xmax}), yrange: ({ymin},{ymax}), zrange: ({zmin},{zmax})')

In [ ]:
# Rounding min and max to nearest 100 away from 0
# x0 = np.arange(-4000,4500,50)
# x1 = np.arange(-2300,3100,50)
# X = np.stack(np.meshgrid(x0,x1,indexing='ij'),-1)

# Use a subset of the atlas coordinates
x0 = np.arange(roundCoord(atlas_y,np.min(ycoords)),roundCoord(atlas_y,np.max(ycoords)),20)
x1 = np.arange(roundCoord(atlas_z,np.min(zcoords)),roundCoord(atlas_z,np.max(zcoords)),20)

X = np.stack(np.meshgrid(x0,x1,indexing='ij'),-1)

## Step 2: Estimate local cell density

In [ ]:
density = np.zeros_like(X[...,0], dtype = 'float64')
sigma = 100
for coord in x:
    density += np.exp(  np.sum(-(X - coord)**2/2/sigma**2, -1 ) )/np.sqrt(2.0*np.pi*sigma**2)**2

fig,ax = plt.subplots()

# Plots 
h = ax.imshow(density)
plt.colorbar(h)

# note that the y axis is flipped by default, relative to what we sa
ax.invert_yaxis()

ax.set_xlabel('z (Dorsal => Ventral)')
ax.set_ylabel('y')
ax.set_title(f'Local density of cells')

## Step 3: Estimate local property averages

In [ ]:
local_avg = np.zeros_like(X[...,0], dtype = 'float64')
for xi,pi in zip(x,prop):
    local_avg += np.exp(  np.sum(-(X - xi)**2/2/sigma**2, -1 ) )/np.sqrt(2.0*np.pi*sigma**2)**2 * pi

# local_avg is actually a local sum until we divide by density
local_avg = local_avg / density

fig,ax = plt.subplots()
# ax.scatter(zcoords, ycoords)
h = ax.imshow(local_avg)

plt.colorbar(h)
ax.invert_yaxis()
ax.set_title(f'Local average of {prop_name}')

# ax.set_xticks(atlas_z, minor = True)
# ax.set_yticks(atlas_y, minor = True)

# ax.set_xbound(-2500,2500)
# ax.set_ybound(-4500,4500)

## Step 4: Render local_avg map with transparency

In [ ]:
cm = plt.cm.inferno
RGBA = cm(local_avg/np.max(local_avg))

fig,ax = plt.subplots()
ax.imshow(RGBA)
ax.invert_yaxis()

### Step 4.1: Set map to 0 when far away from cells

In [ ]:
RGBA[...,-1] = density/np.max(density)
fig,ax = plt.subplots()
h = ax.imshow(RGBA,cmap=cm,vmin=np.min(local_avg),vmax=np.max(local_avg)) # extra arguments are for making a colorbar
ax.invert_yaxis()
# we need a colorbar
fig.colorbar(h)

## Step 4.2: Put atlas behind heatmap

In [ ]:
fig,ax = plt.subplots()

# Plots 
h = ax.imshow(RGBA, cmap=cm,vmin=np.min(local_avg),vmax=np.max(local_avg), extent = (np.min(x1),np.max(x1),np.min(x0),np.max(x0))) # 'extent': Defines area which 'density' will fill
plt.colorbar(h)

# note that the y axis is flipped by default, relative to what we sa
ax.invert_yaxis()

ax.set_xlabel('z (Dorsal => Ventral)')
ax.set_ylabel('y')
ax.set_title(f'Local avg of {prop_name}')

# Changes axes labels from idx of 'density' to (z,y) coords from atlas
ax.set_xticks(atlas_z, minor = True)
ax.set_yticks(atlas_y, minor = True)
ax.minorticks_off()

# Example in 3D

## Step 1: Define voxel grid

In [ ]:
x = zip(ycoords, zcoords, xcoords)

# Rounding min and max to nearest 100 away from 0
x0 = np.arange(-4000,4500,50)
x1 = np.arange(-2300,3100,50)
x2 = np.arange(-1150,2850,50) # Padded each size by 50 => len(x2) = 80, better number for visualizations
X = np.stack(np.meshgrid(x0,x1,x2,indexing='ij'),-1)

## Step 2: Estimate local cell density

In [ ]:
density = np.zeros_like(X[...,0], dtype = 'float64')
sigma = 100
count = 0
for coord in x:
    density += np.exp(  np.sum(-(X - coord)**2/2/sigma**2, -1 ) )/np.sqrt(2.0*np.pi*sigma**2)**3
    if count % 100 == 0:
        print(f'Finished {count}/{len(xcoords)}')
    count+=1

In [ ]:
# Generate data for plots
num_subplots = 16
density_plots = np.zeros([np.shape(density)[0], np.shape(density)[1], num_subplots]) # Will contain all density MIPS for the subplot figure below
count = 0
nplots = 5 # Number of plots combined in MIP

for i in np.arange(np.shape(density_plots)[2]): # Iterate over each elem of 'density_plots'
    # print(f'=== {i} ===')
    for j in np.arange(count*nplots,(count+1)*nplots): # Iterate over the ith set of 5 elements in 'density'
        # print(f'{j}: {len(np.unique(density_plots[:,:,i]))}, {np.sum(density_plots[:,:,i])}')
        density_plots[:,:,i] = np.maximum(density_plots[:,:,i],density[:,:,j])
    count+=1

In [ ]:
fig, ax = plt.subplots(4,4,sharey=True,sharex=True)
x_tick = 50
x_start = -1150

count = 0
for ai in ax:
    for aj in ai:
        im = aj.imshow(density_plots[:,:,count],vmin=np.min(density_plots),vmax=np.max(density_plots))
        aj.set_title(f'{count*nplots*x_tick+x_start}:{(count+1)*nplots*x_tick-1+x_start}')
        count+=1

fig.set_size_inches(8,8)
fig.subplots_adjust(wspace=0)
fig.suptitle('Local cell densities')
fig.colorbar(im,ax=ax.ravel().tolist())
fig.supxlabel('z (Dorsal => Ventral)')
fig.supylabel('y')

# Title: The range of x coordinates in the MIP

## Step 3: Estimate local property average

In [ ]:
density[density == 0.0] = np.unique(density)[1] # Set all instances of 0.0 to the second lowest value from 'density'

# Gets rid of divide by 0 error with minimal manipulation of data

In [ ]:
local_avg = np.zeros_like(X[...,0], dtype = 'float64')
count = 0
for yi,zi,xi,pi in zip(ycoords,zcoords,xcoords,prop):
    local_avg += np.exp(  np.sum(-(X - [yi,zi,xi])**2/2/sigma**2, -1 ) )/np.sqrt(2.0*np.pi*sigma**2)**3 * pi
    if count % 100 == 0:
        print(f'Finished {count}/{len(xcoords)}')
    count+=1

# local_avg is actually a local sum until we divide by density
local_avg = local_avg / density

In [ ]:
# Generate data for plots
num_subplots = 16
local_avg_plots = np.zeros([np.shape(local_avg)[0], np.shape(local_avg)[1], num_subplots]) # Will contain all density MIPS for the subplot figure below
count = 0
nplots = 5

for i in np.arange(np.shape(local_avg_plots)[2]): # Iterate over each elem of 'density_plots'
    # print(f'=== {i} ===')
    for j in np.arange(count*nplots,(count+1)*nplots): # Iterate over the ith set of 5 elements in 'density'
        # print(f'{j}: {len(np.unique(density_plots[:,:,i]))}, {np.sum(density_plots[:,:,i])}')
        local_avg_plots[:,:,i] = np.maximum(local_avg_plots[:,:,i],local_avg[:,:,j])
    count+=1

In [ ]:
fig, ax = plt.subplots(4,4,sharey=True,sharex=True)
x_tick = 50
x_start = -1150

count = 0
for ai in ax:
    for aj in ai:
        im = aj.imshow(local_avg_plots[:,:,count])
        aj.set_title(f'{count*nplots*x_tick+x_start}:{(count+1)*nplots*x_tick-1+x_start}')
        count+=1

fig.set_size_inches(8,8)
fig.subplots_adjust(wspace=0)
fig.suptitle(f'{prop_name}')
fig.colorbar(im,ax=ax.ravel().tolist())
fig.supxlabel('z (Dorsal => Ventral)')
fig.supylabel('y')

# Title: The range of x coordinates in the MIP